In [1]:
# Import 
import os
import pandas as pd
import regex as re
from pathlib import Path
from collections import Counter

In [8]:
# Read in the sample text
with open('Beispieltexte/Briefausweilbeispieltext.txt') as f:
    text = f.read()
    print(text)

Aber meine Schwiegersöhne sind flott, gelt du? Man weiß nicht, welches daß der Liebere ist! Saukerle, alle miteinander! Das heißt, vom Albert weiß man nichts anderes, als daß er den ganzen Aufsichtsrat gestimmt hat zur Vertrauensseligkeit,
aber das ist Haufen genug! Und, nicht wahr, mit Rechtem ist doch auch der Albert nicht zu seinem Millionenbesitz gelangt. Vier Villen hat der Kauz: – eine in Flüelen, eine in Menaggio, eine in Lauterbrunnen, eine bei Zürich. Doch halt – hatte muß es heißen. Ob er heut noch 's Dach überm Kopf hat – wer kann's sagen.

Der Léon soll sich fortgemacht haben, denk auch! Doch heißt's, es sei ihm nichts anzuhaben. Jetzt – was so ein flüchtiges Bankdirektorshirn ausbrüten kann, der Léon wird's ausbrüten, und der Herr Aufsichtsrat wird ihm schon soufflieren, wo er stecken bleibt; gib Obacht! 's ist halt sehr verdächtig.

Dein gebeugter Vater.

Adele an ihre Schwester Josefine

Privatim und in Eile.

Geliebte teure Fifi!

Eine arge Komplikation in Léons Geschäf

In [10]:
# Print how many characters are in the sample text
text_length = len(text)
print(text_length)

28266


In [12]:
# Text preparation steps:

single_spaces_text = re.sub('\n+', '\n', text)
single_spaces_text = re.sub('\s+', ' ', single_spaces_text)
single_spaces_text = re.sub('--+', ' -- ', single_spaces_text)
single_spaces_text = re.sub('&', 'und', single_spaces_text)
single_spaces_text = re.sub('<|/', ' ', single_spaces_text)
single_spaces_text = re.sub('>', ' ', single_spaces_text)
single_spaces_text = re.sub("'s", ' es', single_spaces_text)
single_spaces_text = re.sub(';', '.', single_spaces_text)
single_spaces_text = re.sub('\d', ' ', single_spaces_text)
text = re.sub('\s+', ' ', single_spaces_text)

print(text[:200])

Aber meine Schwiegersöhne sind flott, gelt du? Man weiß nicht, welches daß der Liebere ist! Saukerle, alle miteinander! Das heißt, vom Albert weiß man nichts anderes, als daß er den ganzen Aufsichtsrat gestimmt hat zur Vertrauensseligkeit, aber das ist Haufen genug! Und, nicht wahr, mit Rechtem ist doch auch der Albert nicht zu seinem Millionenbesitz gelangt. Vier Villen hat der Kauz: – eine in Flüelen, eine in Menaggio, eine in Lauterbrunnen, eine bei Zürich. Doch halt – hatte muß es heißen. Ob er heut noch es Dach überm Kopf hat – wer kann es sagen. Der Léon soll sich fortgemacht haben, denk auch! Doch heißt es, es sei ihm nichts anzuhaben. Jetzt – was so ein flüchtiges Bankdirektorshirn ausbrüten kann, der Léon wird es ausbrüten, und der Herr Aufsichtsrat wird ihm schon soufflieren, wo er stecken bleibt. gib Obacht! es ist halt sehr verdächtig. Dein gebeugter Vater. Adele an ihre Schwester Josefine Privatim und in Eile. Geliebte teure Fifi! Eine arge Komplikation in Léons Geschäften

In [38]:
# Liste von Briefanfängen und -enden
letter_openings = [
    r'[»]?Mein[e]? [lL]iebe[rsn]?\s[A-Za-z]*[.!,]?',
    r'[»]?Hochverehrte[rsn]?\s[A-Za-z]*[.!,]?',
    r'[»]?Einziggeliebt[er]?\s[A-Za-z]*[.!,]?',
    r'[»]?Geehrte[rsn]?\s[A-Za-z]*[.!,]?',
    r'[»]?Sehr geehrte[srn]?\s[A-Za-z]*[.!,]?',
    r'[»]?Sehr verehrte[rs]?\s[A-Za-z]*[.!,]?',
    r'[»]?Grüss dich\s[A-Za-z]*[.!,]?',
    r'[»]?Grüß dich\s[A-Za-z]*[.!,]?',
    r'[»]?\b(?!die\s+)Liebe\b\s[A-Za-z]*[.!,]?',
    #r'[»]?Liebe[sr]+\s[A-Za-z]*[.!,]?',
    r'»Liebe[sr]+\s[A-Z][a-z\s]*[A-Za-z.!,]*',
    r'»Liebste[sr]?\s[A-Za-z]*[A-Za-z.!,]*',
    r'[»]?Lieber Vater[.!,]?',
    r'[»]?Liebster Vater[.!,]?',
    r'[»]?Liebe Mutter[.!,]?',
    r'[»]?Liebste Mutter[.!,]?',
    r'[»]?Lieber Freund[.!,]?',
    r'[»]?Geliebteste[r]?[.!,]?',
    r'[»]?Geliebte[r]?[.!,]?',
    r'[»]?Einzig geliebteste[r]?[.!,]?',
    r'[»]?Einzig geliebte[r]?[.!,]?',
    r'[»]?Hochverehrter Herr[.!,]?',
    r'[»]?Hochverehrte Frau[.!,]?',
    r'[»]?Werte[rs]?\s[A-Za-z]*[.!,]?',
    r'[»]?Mein geliebte[rs]?\s[A-Za-z]*[.!,]?',
    r'[»]?Teuerste[sr]?\s[A-Za-z]*[.!,]?',
    r'[»]?Teure[sr]?\s[A-Za-z]*[.!,]?',
    r'[»]?Mein Liebchen[.!,]?',
    r'(?:[A-ZÄÖÜa-zäöüß\s]+,\s*den\s+\d+\.\s+[A-ZÄÖÜa-zäöüß]+\s+\d{4}\.)',
    # Weitere Anfangsformulierungen hier
]


In [45]:
# Iterate over each pattern
for pattern in letter_openings:
    # Initialize an empty list to store matches for the current pattern
    pattern_matches = []
    
    # Iterate over matches for the current pattern and append them to the list
    for match in re.finditer(pattern, text):
        match_text = match.group(0)  # Get the matched text
        pattern_matches.append(match_text)
    
    # Store the list of matches in the extracted_matches dictionary
    extracted_matches[pattern] = pattern_matches

# Print the extracted matches without indicating the pattern
for pattern_matches in extracted_matches.values():
    for match_text in pattern_matches:
        print(f"'{match_text}'")

'Einziggeliebte Josy!'
'»Geehrter Herr'
'Sehr geehrter Herr!'
'Sehr geehrte Frau!'
'Sehr geehrter Herr'
'Sehr geehrte Frau!'
'Liebe Schwester!'
'Liebe arme'
'Liebe Adele!'
'Liebe Schwester'
'»Liebe Adelaide!'
'»Lieber Hans!'
'Geliebte'
'Einzig geliebteste'
'Einzig geliebte'
'Teure Schwester!'


In [40]:
# Initialize an empty dictionary to store the matches
extracted_matches = {}

# Iterate over each pattern
for pattern in letter_openings:
    # Initialize an empty list to store matches for the current pattern
    pattern_matches = []
    
    # Iterate over matches for the current pattern and append them to the list
    for match in re.finditer(pattern, text):
        match_text = match.group(0)  # Get the matched text
        match_start = match.start()  # Get the start index of the match
        match_end = match.end()      # Get the end index of the match
        
        # Append the match, its start index, and end index to the list as a tuple
        pattern_matches.append((match_text, match_start, match_end))
    
    # Store the list of matches in the extracted_matches dictionary under the pattern name
    extracted_matches[pattern] = pattern_matches

# Print the extracted matches with index indication
for pattern, pattern_matches in extracted_matches.items():
    for i, (match_text, match_start, match_end) in enumerate(pattern_matches, start=1):
        #print(f"Pattern: '{pattern}', Match {i}")
        print(f"'{match_text}'")
        print(f"Start Index: {match_start}, End Index: {match_end}")

'Einziggeliebte Josy!'
Start Index: 3876, End Index: 3896
'»Geehrter Herr'
Start Index: 15001, End Index: 15015
'Sehr geehrter Herr!'
Start Index: 5556, End Index: 5575
'Sehr geehrte Frau!'
Start Index: 5782, End Index: 5800
'Sehr geehrter Herr'
Start Index: 7048, End Index: 7066
'Sehr geehrte Frau!'
Start Index: 7408, End Index: 7426
'Liebe Schwester!'
Start Index: 2627, End Index: 2643
'Liebe arme'
Start Index: 2967, End Index: 2977
'Liebe Adele!'
Start Index: 4707, End Index: 4719
'Liebe Schwester'
Start Index: 6260, End Index: 6275
'»Liebe Adelaide!'
Start Index: 11872, End Index: 11888
'»Lieber Hans!'
Start Index: 20818, End Index: 20831
'Geliebte'
Start Index: 937, End Index: 945
'Einzig geliebteste'
Start Index: 1464, End Index: 1482
'Einzig geliebte'
Start Index: 1464, End Index: 1479
'Teure Schwester!'
Start Index: 3345, End Index: 3361


In [41]:

# Initialize an empty dictionary to store the matches with their indices and following 100 characters
extracted_matches = {}

# Iterate over each pattern
for pattern in letter_openings:
    # Initialize an empty list to store matches for the current pattern
    pattern_matches = []
    
    # Iterate over matches for the current pattern and append them to the list
    for match in re.finditer(pattern, text):
        match_text = match.group(0)  # Get the matched text
        match_start = match.start()  # Get the start index of the match
        match_end = match.end()      # Get the end index of the match
        
        # Extract the following 100 characters
        following_text = text[match_end:match_end + 100]
        
        # Append the match, its start index, end index, and following text to the list as a tuple
        pattern_matches.append((match_text, match_start, match_end, following_text))
    
    # Store the list of matches in the extracted_matches dictionary under the pattern name
    extracted_matches[pattern] = pattern_matches

# Print the extracted matches with index indication and following text
for pattern, pattern_matches in extracted_matches.items():
    for i, (match_text, match_start, match_end, following_text) in enumerate(pattern_matches, start=1):
        #print(f"Pattern: '{pattern}', Match {i}")
        print(f"'{match_text}'")
        print(f"Start Index: {match_start}, End Index: {match_end}")
        print(f"Following 100 Characters: '{following_text}'")

'Einziggeliebte Josy!'
Start Index: 3876, End Index: 3896
Following 100 Characters: ' Was soll Albert anfangen, wenn du nicht willst! Wir glaubten, du seiest nicht so hart wie die übrig'
'»Geehrter Herr'
Start Index: 15001, End Index: 15015
Following 100 Characters: ' Hendlbadschi! Ihre Tragödie »Das Blutgericht« habe ich erhalten und sofort gelesen. Es ist mir ein '
'Sehr geehrter Herr!'
Start Index: 5556, End Index: 5575
Following 100 Characters: ' Mein auf übermorgen festgesetzter Vortrag muß leider verschoben werden, da ich verreisen muß. Bitte'
'Sehr geehrte Frau!'
Start Index: 5782, End Index: 5800
Following 100 Characters: ' Bitte, erschrecken Sie nicht, wenn morgen Fräulein Dr. Lauterer statt meiner bei Ihnen Besuch macht'
'Sehr geehrter Herr'
Start Index: 7048, End Index: 7066
Following 100 Characters: ' Direktor! Hierdurch bitte ich Sie um die Erlaubnis, meine Tochter Rösi schon jetzt, acht Tage vor B'
'Sehr geehrte Frau!'
Start Index: 7408, End Index: 7426
Following 100 Char

In [46]:
# Initialize an empty dictionary to store the matches and their following 100 characters
extracted_matches = {}

# Iterate over each pattern
for pattern in letter_openings:
    # Initialize an empty list to store matches with following text for the current pattern
    pattern_matches = []
    
    # Iterate over matches for the current pattern and append them to the list
    for match in re.finditer(pattern, text):
        match_text = match.group(0)  # Get the matched text
        match_end = match.end()      # Get the end index of the match
        
        # Extract the following 100 characters
        following_text = text[match_end:match_end + 100]
        
        # Combine the match and its following 100 characters and append to the list
        combined_text = match_text + following_text
        pattern_matches.append(combined_text)
    
    # Store the list of matches with following text in the extracted_matches dictionary under the pattern name
    extracted_matches[pattern] = pattern_matches

# Print the extracted matches and their following 100 characters
for pattern, pattern_matches in extracted_matches.items():
    for i, combined_text in enumerate(pattern_matches, start=1):
        #print(f"Pattern: '{pattern}', Match {i}")
        print(f"'{combined_text}'")


'Einziggeliebte Josy! Was soll Albert anfangen, wenn du nicht willst! Wir glaubten, du seiest nicht so hart wie die übrig'
'»Geehrter Herr Hendlbadschi! Ihre Tragödie »Das Blutgericht« habe ich erhalten und sofort gelesen. Es ist mir ein '
'Sehr geehrter Herr! Mein auf übermorgen festgesetzter Vortrag muß leider verschoben werden, da ich verreisen muß. Bitte'
'Sehr geehrte Frau! Bitte, erschrecken Sie nicht, wenn morgen Fräulein Dr. Lauterer statt meiner bei Ihnen Besuch macht'
'Sehr geehrter Herr Direktor! Hierdurch bitte ich Sie um die Erlaubnis, meine Tochter Rösi schon jetzt, acht Tage vor B'
'Sehr geehrte Frau! Wir wenden uns an Sie mit unserer Antwort auf eine Anfrage, die vor ungefähr einem Monat an unsere '
'Liebe Schwester! Ich weiß nicht, ob es Léon bekannt ist, daß Albert dasselbe Gesuch an uns stellt wie dein Mann. Es '
'Liebe arme Marie! Ich weiß nicht, ob dein Albert hier mit Léon zusammentreffen, oder ob er sich auch vor ihm v'
'Liebe Adele! Mitfolgend den Hausschlüssel 

In [47]:
#Beste Lösung!

# Initialize an empty dictionary to store the matches, their start indices, and following 100 characters
extracted_matches = {}

# Iterate over each pattern
for pattern in letter_openings:
    # Initialize an empty list to store matches with start indices and following text for the current pattern
    pattern_matches = []
    
    # Iterate over matches for the current pattern and append them to the list
    for match in re.finditer(pattern, text):
        match_text = match.group(0)  # Get the matched text
        match_start = match.start()  # Get the start index of the match
        match_end = match.end()      # Get the end index of the match
        
        # Extract the following 100 characters
        following_text = text[match_end:match_end + 100]
        
        # Combine the match, its start index, and following 100 characters and append to the list
        combined_text = match_text + following_text
        pattern_matches.append((combined_text, match_start))
    
    # Store the list of matches with start indices and following text in the extracted_matches dictionary under the pattern name
    extracted_matches[pattern] = pattern_matches

# Print the extracted matches, their start indices, and following 100 characters
for pattern, pattern_matches in extracted_matches.items():
    for i, (combined_text, match_start) in enumerate(pattern_matches, start=1):
        #print(f"Pattern: '{pattern}', Match {i}")
        print(f"'{combined_text}'")
        print(f"Start Index: {match_start}")


'Einziggeliebte Josy! Was soll Albert anfangen, wenn du nicht willst! Wir glaubten, du seiest nicht so hart wie die übrig'
Start Index: 3876
'»Geehrter Herr Hendlbadschi! Ihre Tragödie »Das Blutgericht« habe ich erhalten und sofort gelesen. Es ist mir ein '
Start Index: 15001
'Sehr geehrter Herr! Mein auf übermorgen festgesetzter Vortrag muß leider verschoben werden, da ich verreisen muß. Bitte'
Start Index: 5556
'Sehr geehrte Frau! Bitte, erschrecken Sie nicht, wenn morgen Fräulein Dr. Lauterer statt meiner bei Ihnen Besuch macht'
Start Index: 5782
'Sehr geehrter Herr Direktor! Hierdurch bitte ich Sie um die Erlaubnis, meine Tochter Rösi schon jetzt, acht Tage vor B'
Start Index: 7048
'Sehr geehrte Frau! Wir wenden uns an Sie mit unserer Antwort auf eine Anfrage, die vor ungefähr einem Monat an unsere '
Start Index: 7408
'Liebe Schwester! Ich weiß nicht, ob es Léon bekannt ist, daß Albert dasselbe Gesuch an uns stellt wie dein Mann. Es '
Start Index: 2627
'Liebe arme Marie! Ich weiß n

In [44]:
print(len(extracted_matches))

28


In [51]:
print(extracted_matches)

{'[»]?Mein[e]? [lL]iebe[rsn]?\\s[A-Za-z]*[.!,]?': [], '[»]?Hochverehrte[rsn]?\\s[A-Za-z]*[.!,]?': [], '[»]?Einziggeliebt[er]?\\s[A-Za-z]*[.!,]?': [('Einziggeliebte Josy! Was soll Albert anfangen, wenn du nicht willst! Wir glaubten, du seiest nicht so hart wie die übrig', 3876)], '[»]?Geehrte[rsn]?\\s[A-Za-z]*[.!,]?': [('»Geehrter Herr Hendlbadschi! Ihre Tragödie »Das Blutgericht« habe ich erhalten und sofort gelesen. Es ist mir ein ', 15001)], '[»]?Sehr geehrte[srn]?\\s[A-Za-z]*[.!,]?': [('Sehr geehrter Herr! Mein auf übermorgen festgesetzter Vortrag muß leider verschoben werden, da ich verreisen muß. Bitte', 5556), ('Sehr geehrte Frau! Bitte, erschrecken Sie nicht, wenn morgen Fräulein Dr. Lauterer statt meiner bei Ihnen Besuch macht', 5782), ('Sehr geehrter Herr Direktor! Hierdurch bitte ich Sie um die Erlaubnis, meine Tochter Rösi schon jetzt, acht Tage vor B', 7048), ('Sehr geehrte Frau! Wir wenden uns an Sie mit unserer Antwort auf eine Anfrage, die vor ungefähr einem Monat an uns

In [52]:
import csv

# Sample extracted_matches dictionary (replace this with your actual dictionary)
#extracted_matches = {
#    'Pattern1': [('Match1', 0), ('Match2', 15)],
#    'Pattern2': [('Match3', 5)]
#}

# Specify the CSV file path
csv_file_path = "letter_openings_sample_text.csv"

# Write the extracted_matches dictionary to the CSV file
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    
    # Write a header row (optional)
    writer.writerow(['Pattern', 'Detected Text', 'Start Index'])
    
    # Iterate through the extracted_matches dictionary and write each match to the CSV file
    for pattern, matches in extracted_matches.items():
        for match_text, match_start in matches:
            writer.writerow([pattern, match_text, match_start])

print(f"The dictionary has been saved to '{csv_file_path}'.")


The dictionary has been saved to 'letter_openings_sample_text.csv'.


In [ ]:
End of the Notebook.